# Задание

### Разработать простой тест на корректность расчёта баланса

Для начала подключимся к базе данных, для этого будем использовать библиотеку `pyodbc`, предназначенную для работы с базами данных ODBC (например, MS SQL Server)

In [1]:
import pyodbc 

str_param = (
    r'Driver={ODBC Driver 18 for SQL Server};'
    r'Server=localhost;'
    r'Database=PaymentData;'
    r'Trusted_Connection=yes;'
    r'Encrypt=no;'
    )
cnxn = pyodbc.connect(str_param)
cursor = cnxn.cursor()

Для проведения теста будем использовать пример из `Balance description.pdf`. В нём производится 4 платежа:

1. `Bank` -> `Supplier` - "Авансовый платёж" на 400000

2. `Supplier` -> `Client` - "Закупка материалов" на 100000

3. `Client` -> `Cashbox` - "На закупку материалов" на 150000
    
4. `Cashbox` -> `Bank` - "Возврат кредита" на 100000

Для этого нужно:

1. Получить `Oid` для `Bank`, `Supplier`, `Client`, `Cashbox`, `Project` (вытекает из схемы базы данных, см. ниже)

![Схема используемой части БД](scheme.png)

2. Узнать `Oid` для категории каждого платежа

3. Провести платежи, при этом каждый раз проверяя правильность баланса

### №1. Получение `Oid` 

In [2]:
def get_oid(table):
    query = f"""
    SELECT TOP 1 {table}.Oid
    FROM {table} 
    INNER JOIN PaymentParticipant ON {table}.Oid = PaymentParticipant.Oid
    ORDER BY NEWID()
    """
    return cursor.execute(query).fetchval()


def get_proj_oid(client_oid):
    query = f"""
    SELECT Oid
	FROM Project
	WHERE Client = '{client_oid}'
    """
    return cursor.execute(query).fetchval()


bank_oid = get_oid('Bank')
supplier_oid = get_oid('Supplier')
client_oid = get_oid('Client')
cashbox_oid = get_oid('Cashbox')
project_oid = get_proj_oid(client_oid)

### №2. Получение `Oid` для категории каждого платежа

In [3]:
def get_category_oid(category):
    query = f"""
    SELECT Oid FROM PaymentCategory
    WHERE Name = '{category}'
    """ 
    return cursor.execute(query).fetchval()

### №3. Проведение платежей и проверка баланса

Функция для получения баланса

In [4]:
def get_balance(oid):
    query = f"""
    SELECT Balance FROM PaymentParticipant 
    WHERE Oid = '{oid}'
    """
    return cursor.execute(query).fetchval()

Балансы до операции

In [5]:
def print_balances(bank_oid, supplier_oid, client_oid, cashbox_oid):
    print(f'Bank balance: {get_balance(bank_oid)}')
    print(f'Supplier balance: {get_balance(supplier_oid)}')
    print(f'Client balance: {get_balance(client_oid)}')
    print(f'Cashbox balance: {get_balance(cashbox_oid)}')

In [6]:
print_balances(bank_oid, supplier_oid, client_oid, cashbox_oid)

Bank balance: -10800
Supplier balance: 13987
Client balance: -18773
Cashbox balance: 8924


Проведение первого платежа

In [7]:
first_payment_oid = get_category_oid("Аванс на материалы")
query = f"""
INSERT INTO Payment
(Oid, Amount, Category, Project, Justification,
Comment, Date, Payer, Payee, OptimisticLockField, GCRecord,
CreateDate, CheckNumber, IsAuthorized, Number)
VALUES
(NEWID(), 400000, '{first_payment_oid}', '{project_oid}', NULL,
'Первый платёж', NULL, '{bank_oid}', '{supplier_oid}', NULL, NULL,
NULL, NULL, NULL, NULL)
"""
cursor.execute(query)
cursor.commit()

Проверка баланса

In [8]:
print_balances(bank_oid, supplier_oid, client_oid, cashbox_oid)

Bank balance: -410800
Supplier balance: 413987
Client balance: -18773
Cashbox balance: 8924


Проведение второго платежа

In [9]:
second_payment_oid = get_category_oid("Закупка материалов")
query = f"""
    INSERT INTO Payment
    (Oid, Amount, Category, Project, Justification,
	Comment, Date, Payer, Payee, OptimisticLockField, GCRecord,
	CreateDate, CheckNumber, IsAuthorized, Number)
    VALUES
    (NEWID(), 100000, '{second_payment_oid}', '{project_oid}', NULL,
    'Второй платёж', NULL, '{supplier_oid}', '{client_oid}', NULL, NULL,
    NULL, NULL, NULL, NULL)
    """
cursor.execute(query)
cursor.commit()

Проверка баланса

In [10]:
print_balances(bank_oid, supplier_oid, client_oid, cashbox_oid)

Bank balance: -410800
Supplier balance: 313987
Client balance: 81227
Cashbox balance: 8924


Проведение третьего платежа

In [11]:
third_payment_oid = get_category_oid("Закупка материалов")
query = f"""
    INSERT INTO Payment
    (Oid, Amount, Category, Project, Justification,
	Comment, Date, Payer, Payee, OptimisticLockField, GCRecord,
	CreateDate, CheckNumber, IsAuthorized, Number)
    VALUES
    (NEWID(), 150000, '{third_payment_oid}', '{project_oid}', NULL,
    'Третий платёж', NULL, '{client_oid}', '{cashbox_oid}', NULL, NULL,
    NULL, NULL, NULL, NULL)
    """
cursor.execute(query)
cursor.commit()

Проверка баланса

In [12]:
print_balances(bank_oid, supplier_oid, client_oid, cashbox_oid)

Bank balance: -410800
Supplier balance: 313987
Client balance: -68773
Cashbox balance: 158924


Проведение четвёртого платежа

In [13]:
payment_category_oid = get_category_oid("Возврат кредита")
query = f"""
    INSERT INTO Payment
    (Oid, Amount, Category, Project, Justification,
	Comment, Date, Payer, Payee, OptimisticLockField, GCRecord,
	CreateDate, CheckNumber, IsAuthorized, Number)
    VALUES
    (NEWID(), 100000, '{payment_category_oid}', '{project_oid}', NULL,
    'Частичное погашение кредита ', NULL, '{cashbox_oid}', '{bank_oid}', NULL, NULL,
    NULL, NULL, NULL, NULL)
    """
cursor.execute(query)
cursor.commit()

Финальная проверка баланса

In [14]:
print_balances(bank_oid, supplier_oid, client_oid, cashbox_oid)

Bank balance: -310800
Supplier balance: 313987
Client balance: -68773
Cashbox balance: 58924


In [15]:
cursor.close()
cnxn.close()

Использованные источники:
1. https://github.com/mkleehammer/pyodbc/wiki
2. https://habr.com/ru/articles/321510/